<a href="https://colab.research.google.com/github/amirmohammadkalateh/Mental_Health_Lifestyle/blob/main/pytorch_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load and preprocess data
df = pd.read_csv('Mental_Health_Lifestyle_Dataset.csv')

# Encode categorical variables
categorical_cols = ['Country', 'Gender', 'Exercise Level', 'Diet Type', 'Stress Level', 'Mental Health Condition']
label_encoders = {}
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

# Prepare features and target
X = df.drop(['Happiness Score'], axis=1)
y = df['Happiness Score']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test.values).reshape(-1, 1)

# Define the model
class MentalHealthANN(nn.Module):
    def __init__(self, input_size):
        super(MentalHealthANN, self).__init__()
        self.layer1 = nn.Linear(input_size, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# Initialize model and optimizer
model = MentalHealthANN(X_train.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training
epochs = 100
batch_size = 32
n_batches = len(X_train_tensor) // batch_size

print("Training the model...")
for epoch in range(epochs):
    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size

        optimizer.zero_grad()
        outputs = model(X_train_tensor[start:end])
        loss = criterion(outputs, y_train_tensor[start:end])
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = criterion(y_pred, y_test_tensor)
    print(f'\nTest Loss: {test_loss.item():.4f}')

    # Calculate R-squared
    y_mean = torch.mean(y_test_tensor)
    ss_tot = torch.sum((y_test_tensor - y_mean) ** 2)
    ss_res = torch.sum((y_test_tensor - y_pred) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    print(f'R-squared Score: {r2.item():.4f}')


Training the model...
Epoch [10/100], Loss: 6.4078
Epoch [20/100], Loss: 6.2597
Epoch [30/100], Loss: 6.0097
Epoch [40/100], Loss: 5.5823
Epoch [50/100], Loss: 5.1569
Epoch [60/100], Loss: 4.7947
Epoch [70/100], Loss: 4.4931
Epoch [80/100], Loss: 4.1154
Epoch [90/100], Loss: 3.8323
Epoch [100/100], Loss: 3.7453

Test Loss: 8.3636
R-squared Score: -0.2355
